In [32]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Import Dataset
file_path=r'C:\Users\elvis\OneDrive\Documents\NEU STUDIES FOLDER\Practice File\Invoice\dataset.csv\dataset.csv'
file_path2=r'C:\Users\elvis\OneDrive\Documents\NEU STUDIES FOLDER\Practice File\Invoice\dataset.csv\synthetic_invoice_data.xls'

invoice_org=pd.read_csv(file_path)
invoice_syn=pd.read_csv(file_path2)

# Merge using 'customer_order_id' as key
invoice = pd.merge(invoice_org, invoice_syn, on='CUSTOMER_ORDER_ID', how='left')

print(f' Head: {invoice.head()}')
print(f' Shape: {invoice.shape}')
print(f' Dimension: {invoice.ndim}')
print(f' Structure: {invoice.info()}')
print(f' Summary Statistics: {invoice.describe()}')
print(f' Column Names: {invoice.columns}')
print(f' Missing Values: {invoice.isnull(). sum()}')
print(f' Duplicate: {invoice.duplicated().sum()}')

 Head:    CUSTOMER_ORDER_ID  SALES_ORG      DISTRIBUTION_CHANNEL      DIVISION  \
0          946851639       3537  United States of America  South-Region   
1          963432061       3449                Martinique  South-Region   
2          971991639       3238                   Moldova  South-Region   
3          754349803       3911      United Arab Emirates  South-Region   
4          930253442       2381                    Greece  South-Region   

  RELEASED_CREDIT_VALUE PURCHASE_ORDER_TYPE  COMPANY_CODE  \
0                  0,00                1000          3220   
1                  0,00                1000          3220   
2              82342,02                I200          3260   
3               1471,24                N000          3290   
4                  0,00                N000          3290   

   ORDER_CREATION_DATE_x  ORDER_CREATION_TIME CREDIT_CONTROL_AREA  ...  \
0               20220101                43012                SR02  ...   
1               20220101   

In [33]:
# Standardizing the names
invoice.columns = invoice.columns.str.strip().str.lower().str.replace(' ', '_')

# Step 1: Drop all columns that end with '_y'
invoice = invoice[[col for col in invoice.columns if not col.endswith('_y')]]

# Step 2: Rename remaining columns to remove '_x'
invoice.columns = [col.replace('_x', '') for col in invoice.columns]


print(f' Column Names: {invoice.columns}')
#Convert Data Types(Date and amount)
invoice['order_creation_date'] = pd.to_datetime(invoice['order_creation_date'], format='%Y%m%d', errors='coerce')

invoice['requested_delivery_date'] = pd.to_datetime(invoice['requested_delivery_date'], format='%Y%m%d', errors='coerce')

# Convert 'order_amount' to string before applying string methods
invoice['order_amount'] = invoice['order_amount'].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
invoice['order_amount'] = pd.to_numeric(invoice['order_amount'], errors='coerce')


invoice['released_credit_value'] = invoice['released_credit_value'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
invoice['released_credit_value'] = pd.to_numeric(invoice['released_credit_value'], errors='coerce')



 Column Names: Index(['customer_order_id', 'sales_org', 'distribution_channel', 'division',
       'released_credit_value', 'purchase_order_type', 'company_code',
       'order_creation_date', 'order_creation_time', 'credit_control_area',
       'sold_to_party', 'order_amount', 'requested_delivery_date',
       'order_currency', 'credit_status', 'customer_number',
       'standardized_currency_amount', 'invoice_frequency',
       'outlier_amount_flag', 'duplicate_order_check'],
      dtype='object')


In [34]:
missing_summary = invoice.isnull().sum().sort_values(ascending=False)
print(missing_summary)


credit_status                   886164
duplicate_order_check            53100
outlier_amount_flag              53100
invoice_frequency                53100
standardized_currency_amount     53100
purchase_order_type              18763
order_amount                        32
released_credit_value                1
customer_number                      0
order_currency                       0
requested_delivery_date              0
customer_order_id                    0
sales_org                            0
credit_control_area                  0
order_creation_time                  0
order_creation_date                  0
company_code                         0
division                             0
distribution_channel                 0
sold_to_party                        0
dtype: int64


In [35]:

from sklearn.impute import SimpleImputer

# Separate numeric and categorical
numeric_cols = invoice.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = invoice.select_dtypes(include=['object']).columns

# Impute numeric with mean
num_imputer = SimpleImputer(strategy='mean')
invoice[numeric_cols] = num_imputer.fit_transform(invoice[numeric_cols])

# Impute categorical with most frequent
cat_imputer = SimpleImputer(strategy='most_frequent')
invoice[categorical_cols] = cat_imputer.fit_transform(invoice[categorical_cols])

# 
invoice_cleaned = invoice.copy()



In [36]:
#Feature Engineering

# Days Between Order and Delivery
invoice['order_to_delivery_days'] = (invoice['requested_delivery_date'] - invoice['order_creation_date']).dt.days

#Weekend Order Flag
invoice['is_weekend'] = invoice['order_creation_date'].dt.dayofweek >= 5

# Standardized Currency Amount to USD
currency_rates = {
    'EUR': 1.08,  # example rate
    'RON': 0.22,
    'USD': 1.00,
    'AED':	0.22,
    'CHF': 0.23,
    'CZK':	0.21,
    'GBP':	1.5,
    'PLN':	0.15, 
    'HU1':	0.13,
    'SAR':	0.2,
    'BHD':	0.13,
    'KWD':	0.05,
    'QAR':	0.11,
    'SEK':	0.2,
    'CAD':	0.7,
    'HKD':	0.21,
    'AUD':	0.5,
    'MYR':	0.4,
    'SGD':	0.3,
    'NZD':	0.4 
    
}

invoice['standardized_amount'] = invoice.apply(
    lambda row: row['order_amount'] * currency_rates.get(row['order_currency'], 1),
    axis=1
)

# Duplicate Order Check
invoice['duplicate_order_check'] = invoice.duplicated(
    subset=['order_amount', 'customer_number', 'order_creation_date'], keep=False
)

#Outlier Amount Flag
#Using IQR (Interquartile Range):

Q1 = invoice['order_amount'].quantile(0.25)
Q3 = invoice['order_amount'].quantile(0.75)
IQR = Q3 - Q1

invoice['outlier_amount_flag'] = ((invoice['order_amount'] < (Q1 - 1.5 * IQR)) |
                                  (invoice['order_amount'] > (Q3 + 1.5 * IQR)))
#Credit Released vs Order Amount
invoice['credit_vs_order_ratio'] = invoice['released_credit_value'] / invoice['order_amount']
invoice['high_credit_vs_amount_flag'] = invoice['credit_vs_order_ratio'] > 2  
# If needed: assume missing ratios are 0 (not recommended unless justified)
invoice['credit_vs_order_ratio'].fillna(0, inplace=True)


#Invoice Frequency
invoice['invoice_frequency'] = invoice.groupby('customer_number')['customer_order_id'].transform('count')



C:\Users\elvis\AppData\Local\Temp\ipykernel_19648\4248226739.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  invoice['credit_vs_order_ratio'].fillna(0, inplace=True)


In [37]:
#Rule-Based Fraud Detection
#Duplicate Invoice

invoice['rule_duplicate_invoice'] = invoice.duplicated(
    subset=['customer_number', 'order_amount', 'order_creation_date'], keep=False
).astype(int)

# High Amount Outlier
invoice['rule_outlier_amount'] = invoice['outlier_amount_flag'].astype(int)

#High Credit vs Order Ratio
invoice['rule_high_credit_vs_order'] = invoice['high_credit_vs_amount_flag'].astype(int)

#Short Delivery Time
invoice['rule_short_delivery_time'] = (invoice['order_to_delivery_days'] < 2).astype(int)

#Weekend Invoice Creation
invoice['rule_weekend_order'] = invoice['is_weekend'].astype(int)


#Zero Order Amount with Released Credit
invoice['rule_zero_order_with_credit'] = ((invoice['order_amount'] == 0) & 
                                          (invoice['released_credit_value'] > 0)).astype(int)

#Composite Fraud Score
fraud_cols = [col for col in invoice.columns if col.startswith('rule_')]
invoice['fraud_score'] = invoice[fraud_cols].sum(axis=1)

#Top Risky Invoices
invoice.sort_values(by='fraud_score', ascending=False).head(10)



,customer_order_id,sales_org,distribution_channel,division,released_credit_value,purchase_order_type,company_code,order_creation_date,order_creation_time,credit_control_area,...,standardized_amount,credit_vs_order_ratio,high_credit_vs_amount_flag,rule_duplicate_invoice,rule_outlier_amount,rule_high_credit_vs_order,rule_short_delivery_time,rule_weekend_order,rule_zero_order_with_credit,fraud_score
177550,896361446.0,2786.0,Cape Verde,North-Region,3202.72,NAAA,3290.0,2022-03-25,114507.0,NR01,...,0.0,inf,True,1,0,1,1,0,1,4
253115,844666997.0,2439.0,Bolivia,North-Region,189.21,9999,59.0,2022-05-01,113002.0,NR03,...,0.0,inf,True,0,0,1,1,1,1,4
116985,904772049.0,2921.0,San Marino,North-Region,6990.43,9999,59.0,2022-02-25,114647.0,NR02,...,0.0,inf,True,1,0,1,1,0,1,4
212732,924695690.0,4451.0,French Southern Territories,North-Region,120.22,9999,271.0,2022-04-11,101138.0,SR01,...,0.0,inf,True,1,0,1,1,0,1,4
208127,757470937.0,4055.0,Guyana,North-Region,5912.53,9999,59.0,2022-04-08,73601.0,NR02,...,0.0,inf,True,1,0,1,1,0,1,4
953301,857024774.0,3715.0,Jamaica,South-Region,167.40,X001,3670.0,2022-05-19,100018.0,NR03,...,0.0,inf,True,1,0,1,1,0,1,4
217478,838302586.0,4133.0,French Guiana,North-Region,126.92,9999,271.0,2022-04-12,85852.0,NR03,...,0.0,inf,True,1,0,1,1,0,1,4
535454,892453483.0,4349.0,Maldives,South-Region,0.00,X001,3670.0,2022-02-20,10350.0,SR04,...,3132.0,0.0,False,1,1,0,1,1,0,4
447631,992989345.0,3363.0,South Africa,South-Region,356.40,X001,3670.0,2022-01-31,90022.0,SR01,...,0.0,inf,True,1,0,1,1,0,1,4
447629,787088384.0,2906.0,Botswana,South-Region,579.60,X001,3670.0,2022-01-31,90010.0,SR01,...,0.0,inf,True,1,0,1,1,0,1,4


In [38]:
#Anomaly Detection with Isolation Forest
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

features = [
    'order_amount',
    'released_credit_value',
    'standardized_amount',
    'credit_vs_order_ratio',
    'order_to_delivery_days',
    'invoice_frequency',
]

# Replace NaN and Inf
X = invoice[features].replace([np.inf, -np.inf], np.nan).fillna(0)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Fit Isolation Forest Model
model = IsolationForest(n_estimators=100, contamination=0.02, random_state=42)
invoice['anomaly_score'] = model.fit_predict(X_scaled)

# Tag Suspicious Invoices
invoice['ml_flagged_fraud'] = (invoice['anomaly_score'] == -1).astype(int)

# Count of flagged invoices
print(invoice['ml_flagged_fraud'].value_counts())

# View top anomalies
invoice[invoice['ml_flagged_fraud'] == 1].sort_values(by='fraud_score', ascending=False).head(10)





ml_flagged_fraud
0    1084261
1      22128
Name: count, dtype: int64


,customer_order_id,sales_org,distribution_channel,division,released_credit_value,purchase_order_type,company_code,order_creation_date,order_creation_time,credit_control_area,...,high_credit_vs_amount_flag,rule_duplicate_invoice,rule_outlier_amount,rule_high_credit_vs_order,rule_short_delivery_time,rule_weekend_order,rule_zero_order_with_credit,fraud_score,anomaly_score,ml_flagged_fraud
117712,936979921.0,2954.0,Micronesia,North-Region,9150.76,9999,59.0,2022-02-27,73013.0,NR01,...,True,0,1,1,1,1,0,4,-1,1
445793,770568512.0,2596.0,Algeria,North-Region,0.00,X000,3660.0,2022-01-30,222058.0,NR04,...,False,1,1,0,0,1,0,3,-1,1
297442,811880030.0,3430.0,Marshall Islands,North-Region,38126.78,T001,4200.0,2022-05-24,153120.0,NR03,...,True,1,0,1,0,0,1,3,-1,1
344476,825743096.0,2229.0,British Virgin Islands,South-Region,18900.00,X001,3670.0,2022-01-10,132135.0,SR01,...,False,1,1,0,1,0,0,3,-1,1
344474,934326848.0,4194.0,Belgium,South-Region,18900.00,X001,3670.0,2022-01-10,132131.0,NR02,...,False,1,1,0,1,0,0,3,-1,1
344473,891375928.0,3259.0,Bulgaria,South-Region,18900.00,X001,3670.0,2022-01-10,132126.0,NR02,...,False,1,1,0,1,0,0,3,-1,1
132599,919453874.0,2666.0,French Southern Territories,South-Region,44284.80,9999,108.0,2022-03-05,115020.0,NR01,...,False,0,1,0,1,1,0,3,-1,1
1103897,871682146.0,3650.0,Guadeloupe,North-Region,8819.12,9999,3000.0,2022-05-29,223329.0,NR04,...,False,0,1,0,1,1,0,3,-1,1
1103896,985808149.0,3981.0,Nicaragua,North-Region,11559.19,9999,3000.0,2022-05-29,223013.0,NR04,...,False,0,1,0,1,1,0,3,-1,1
1014252,844751760.0,2383.0,Japan,South-Region,0.00,DFUE,3030.0,2022-01-09,180040.0,SR04,...,False,0,1,0,1,1,0,3,-1,1


In [39]:
#Combine ML + Rule-Based
invoice['final_fraud_flag'] = ((invoice['fraud_score'] >= 3) | (invoice['ml_flagged_fraud'] == 1)).astype(int)
